In [14]:
# import package

import pandas as pd
from selenium import webdriver
import csv
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split

In [15]:
#Compiling the dataframe from available league based dataset

def save_players_in_the_league(league):
    """
    make player data and save
    
    prameter  -----------------------------------------------------
    league : (str) league name
    
    """
    
    # get league team_id team_name dataframe
    league_team_df = pd.read_csv("./league/" + league + ".csv")
    
    # get palyer dataframe function
    def get_player_df(league, team_id, team_name):
        players_df = make_player_info(team_id, team_name)
        players_df.to_csv("./player/" + league + "/" + team_name + ".csv")
        return players_df
    players_df = pd.DataFrame()
    # for one of league teams
    for idx, row in league_team_df.iterrows():
        try_again_num = 0
        

        
        count=len(players_df)
        # there is no player data, try crawling more 3 times
        while len(players_df) == count and try_again_num < 3:
            if try_again_num > 0:
                print("Iterating")
            try_again_num += 1
            players_df_temp = pd.read_csv("./player/" + league + "/" + row.team_name + ".csv")
            
            players_df=players_df.append(players_df_temp)
            
        
    return players_df

In [17]:
#Assimilating data from different leagues, cleaning data by dropping unnecessary features and by eliminating players above the age of 23, transforming the features to a 'per-minute' form

players_df_mine=save_players_in_the_league("PremierLeague")
players_df_mine=players_df_mine.append(save_players_in_the_league("Bundesliga"))
players_df_mine=players_df_mine.append(save_players_in_the_league("Eredivisie"))
players_df_mine=players_df_mine.append(save_players_in_the_league("LaLiga"))
players_df_mine=players_df_mine.append(save_players_in_the_league("Ligue1"))
players_df_mine=players_df_mine.append(save_players_in_the_league("SerieA"))
players_df_mine=players_df_mine.append(save_players_in_the_league("RussiaLeague"))
players_df_mine.rename(columns={'avgp':'avgp1',
                          'ps_y':'avgp',
                          'crosses':'ps',
                               'longb':'crosses',
                               'thrb':'longb'}, 
                 inplace=True)
players_df_mine.drop(columns=['avgp1','flag','position','tall','weight','goals','asists','yel','red','ps_x','motm','rating','tackles','inter','fouls','clear','offsides','drb','blocks','owng','league','team_name'], inplace=True)
players_df_mine['avgp']=(players_df_mine['avgp']*(players_df_mine['half_time']+players_df_mine['full_time']))/players_df_mine['mins']
players_df_mine['crosses']=(players_df_mine['crosses']*(players_df_mine['half_time']+players_df_mine['full_time']))/players_df_mine['mins']
players_df_mine['longb']=(players_df_mine['longb']*(players_df_mine['half_time']+players_df_mine['full_time']))/players_df_mine['mins']
players_df_mine['aw']=(players_df_mine['aw']*(players_df_mine['half_time']+players_df_mine['full_time']))/players_df_mine['mins']
players_df_mine['spg']=(players_df_mine['spg']*(players_df_mine['half_time']+players_df_mine['full_time']))/players_df_mine['mins']
players_df_mine['keyp']=(players_df_mine['keyp']*(players_df_mine['half_time']+players_df_mine['full_time']))/players_df_mine['mins']
players_df_mine['fouled']=(players_df_mine['fouled']*(players_df_mine['half_time']+players_df_mine['full_time']))/players_df_mine['mins']
players_df_mine['off']=(players_df_mine['off']*(players_df_mine['half_time']+players_df_mine['full_time']))/players_df_mine['mins']
players_df_mine['disp']=(players_df_mine['disp']*(players_df_mine['half_time']+players_df_mine['full_time']))/players_df_mine['mins']
players_df_mine['unstch']=(players_df_mine['unstch']*(players_df_mine['half_time']+players_df_mine['full_time']))/players_df_mine['mins']
players_df_mine['label']=0
#setting the list of ace players representing the positive class
labellist={'Bernardo Silva', 'Son Heung-Min', 'Timo Werner','Leroy Sané', 'Bruno Fernandes', 'Hakim Ziyech', 'Mohamed Salah', 'Harry Kane', 'Sadio Mané', 'Paul Pogba', 'Paulo Dybala', 'Marco Verratti', 'Kylian Mbappe-Lottin', 'Philippe Coutinho', 'Sergej Milinkovic-Savic', 'Saúl Ñíguez', 'Mauro Icardi', 'Romelu Lukaku', 'Raheem Sterling', 'Christian Eriksen', 'Dele Alli', 'Marcus Rashford', 'Ousmane Dembele', 'Anthony Martial', 'Mikel Oyarzabal', 'Fabián', 'Neymar', 'Kevin De Bruyne', 'Roberto Firmino', 'Lorenzo Insigne'}
players_df_mine['label']=players_df_mine['name'].apply(lambda x: 1 if x in labellist else 0)
players_df_mine.drop(columns=['full_time','half_time','mins'], inplace=True)
players_train=players_df_mine[(players_df_mine['age']<24) | (players_df_mine['label']==1)]
players_train

Iterating
Iterating


,player_number,name,age,spg,aw,keyp,fouled,off,disp,unstch,avgp,ps,crosses,longb,label
7,125211,Héctor Bellerín,21,0.00444444,0.0122222,0.00777778,0.00222222,0,0.0111111,0.01,0.521111,85.7,0.00555556,0.01,0
15,136824,Alex Iwobi,20,0.0246835,0.00205696,0.0185127,0.00822785,0,0.0246835,0.0329114,0.58212,85.3,0,0.0102848,0
16,125209,Mohamed Elneny,23,0.0166247,0.00692695,0.00415617,0.0138539,0,0.00554156,0.00969773,0.929597,92.8,0.00138539,0.0235516,0
18,95977,Joel Campbell,23,0.0213964,0.00641892,0.0149775,0.0278153,0.00427928,0.0149775,0.029955,0.385135,78.7,0.00213964,0.00641892,0
19,84146,Alex Oxlade Chamberlain,22,0.0237838,0.00951351,0.0214054,0.0142703,0.00237838,0.0261622,0.0261622,0.475676,82.5,0.00951351,0.0285405,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,297202,Dmitri Bogaev,22,0,0.00487805,0,0.00487805,0,0.0097561,0.0292683,0.585366,81.7,0.0097561,0.0292683,0
22,111941,Maxim Palienko,21,0,0,0,0.111111,0,0,0,0.611111,100.0,0,0,0
24,146646,Pavel Dolgov,19,0,0,0,0,0,0.0177778,0.0888889,0.195556,77.8,0,0,0
25,119678,Andrey Ivanov,21,0,0,0,0,0,0,0,3,100.0,0,1,0


In [18]:
#Reading the prediction dataset

final_df=pd.read_csv('predictplayersdataedit.csv')
final_df['aw']=final_df['aw'].astype(float)
final_df['full_time']=final_df['full_time'].astype(int)
final_df['half_time']=final_df['half_time'].astype(int)

In [19]:
#Transforming the prediction dataset

final_df['avgp']=(final_df['avgp']*(final_df['half_time']+final_df['full_time']))/final_df['mins']
final_df['crosses']=(final_df['crosses']*(final_df['half_time']+final_df['full_time']))/final_df['mins']
final_df['longb']=(final_df['longb']*(final_df['half_time']+final_df['full_time']))/final_df['mins']
final_df['aw']=(final_df['aw']*(final_df['half_time']+final_df['full_time']))/final_df['mins']
final_df['spg']=(final_df['spg']*(final_df['half_time']+final_df['full_time']))/final_df['mins']
final_df['keyp']=(final_df['keyp']*(final_df['half_time']+final_df['full_time']))/final_df['mins']
final_df['fouled']=(final_df['fouled']*(final_df['half_time']+final_df['full_time']))/final_df['mins']
final_df['off']=(final_df['off']*(final_df['half_time']+final_df['full_time']))/final_df['mins']
final_df['disp']=(final_df['disp']*(final_df['half_time']+final_df['full_time']))/final_df['mins']
final_df['unstch']=(final_df['unstch']*(final_df['half_time']+final_df['full_time']))/final_df['mins']

In [20]:
y_train=players_train['label']
X_train=players_train.drop(columns=['label','crosses'])
final_df=final_df[X_train.columns]
y_train1=players_train1['label']
X_train1=players_train1.drop(columns=['label','crosses'])

In [21]:
X_train.drop(columns=['player_number', 'name', 'age'], inplace=True)
X_train1.drop(columns=['player_number', 'name', 'age'], inplace=True)

In [22]:
X_predict=final_df.drop(columns=['player_number', 'name', 'age'])

In [23]:
#Training the model and hyperparameter tuing

scaler = MinMaxScaler()
randzo=np.random.choice([0,1], size=1131)
X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled[np.isnan(X_train_scaled)]=0
X_predict_scaled = scaler.fit_transform(X_predict)
X_train_scaled[np.isnan(X_train_scaled)]=0
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_scaled, y_train,random_state=13)
print(y_train_split.sum())
print(y_val_split.sum())
max=0
maxmf=3
maxne=100
for mf in range(2,6):
    for ne in range(10, 130, 10):
        clfrf=RandomForestClassifier(max_features=mf,n_estimators=ne, random_state=0)
        clfrf.fit(X_train_split, y_train_split)
        y_score_lr = clfrf.fit(X_train_split, y_train_split).predict_proba(X_val_split)[:,1]
        fpr_lr, tpr_lr, _ = roc_curve(y_val_split, y_score_lr)
        roc_auc_lr = auc(fpr_lr, tpr_lr)
        if(roc_auc_lr>max):
            max=roc_auc_lr
            maxmf=mf
            maxne=ne
print(max)
print(maxmf)
print(maxne)
clfrfwith=RandomForestClassifier(max_features=maxmf,n_estimators=maxne, random_state=0)
clfrfwith.fit(X_train_split, y_train_split)

25
8
0.8908514492753622
3
50


RandomForestClassifier(max_features=3, n_estimators=50, random_state=0)

In [24]:
#Cross-validating the model

scores = cross_val_score(clfrfwith, X_train_split, y_train_split, cv=10)
scores

array([0.96511628, 0.96511628, 0.97647059, 0.97647059, 0.97647059,
       0.97647059, 0.97647059, 0.96470588, 0.96470588, 0.96470588])

In [25]:
#Results


resultrf=clfrfwith.predict_proba(X_predict_scaled)
result_df=pd.read_csv('predictplayersdataedit.csv')
result_df['aw']=final_df['aw'].astype(float)
result_df['score']=resultrf[:,1]
result_df.sort_values('score', ascending= False).head(50)

,Unnamed: 0,player_number,name,age,mins,spg,keyp,fouled,off,disp,unstch,avgp,ps,crosses,longb,half_time,full_time,aw,score
57,57,353397,Moussa Diaby,20,1362,1.4,1.1,0.9,0.3,1.2,2.8,20.1,78.3,0.7,0.6,9,14,0.001689,0.64
280,280,280691,Matías Vargas,23,942,1.2,1.1,1.1,0.1,2.3,2.1,19.3,70.2,0.6,0.5,6,10,0.001699,0.62
0,0,300713,Kylian Mbappé,21,1516,4.4,2.0,1.1,0.9,1.3,2.4,30.1,80.4,0.4,0.5,3,17,0.002639,0.56
274,274,303158,Yusuf Yazici,23,844,1.2,1.3,0.4,0.1,1.6,0.9,20.4,76.9,1.1,0.8,9,9,0.000000,0.56
206,206,335576,Gabriel Strefezza,23,1403,1.9,1.1,1.8,0.1,1.8,1.4,25.8,82.0,0.5,1.4,4,16,0.002851,0.56
24,24,135891,Allan Saint-Maximin,23,1274,1.7,1.3,1.5,0.1,1.7,2.8,14.5,76.6,0.3,0.5,3,15,0.004239,0.54
101,101,322094,Jonathan Ikoné,22,2075,1.3,1.1,0.9,0.1,1.7,2.6,23.6,78.2,0.1,0.7,2,26,0.004048,0.54
82,82,332398,Nicolò Zaniolo,20,1325,1.9,1.0,2.3,0.2,2.1,2.2,18.9,81.2,0.2,1.2,3,15,0.005434,0.52
1,1,346300,Jadon Sancho,20,1888,1.8,2.2,0.6,0.3,1.0,1.9,47.7,84.6,0.5,0.7,5,21,0.000000,0.52
127,127,363676,Moussa Djenepo,21,926,1.1,0.6,1.7,0.2,1.2,2.2,13.3,74.1,0.2,0.3,8,10,0.003888,0.52
